Baselines Integer Programming for MIS with time limitation based experiments

New: Also gradient based solver has been added

Set Up and Shared Setting

In [2]:
import time
import networkx as nx
from ortools.sat.python import cp_model
import torch
import numpy as np
import pandas as pd
# import gurobipy as gp
# from gurobipy import GRB
from scipy.optimize import minimize
import pulp as pl

In [3]:
n = 500
p = 0.5
time_limit = 5     # seconds per instance
workers   = 192      # parallel threads
instances = 10
m_it = 300

CP-SAT

In [4]:
def solve_mis(n, p, seed, time_limit=50, num_workers=192):
    # 1) Generate graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    
    # 2) Build model
    model = cp_model.CpModel()
    x = [model.NewBoolVar(f'x[{i}]') for i in range(n)]
    for u, v in G.edges():
        model.Add(x[u] + x[v] <= 1)
    model.Maximize(sum(x))
    
    # 3) Configure solver
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    solver.parameters.num_search_workers = num_workers
    solver.parameters.log_search_progress = False  # turn off per-node logs
    
    # 4) Solve and time it
    start = time.time()
    status = solver.Solve(model)
    elapsed = time.time() - start
    
    # 5) Extract result
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        size = sum(1 for i in range(n) if solver.Value(x[i]))
    else:
        size = None
    
    return {
        'seed': seed,
        'status': solver.StatusName(status),
        'size': size,
        'time_sec': elapsed
    }
    
results = []
for seed in range(instances):
    print(f"Solving seed={seed}...", end=' ', flush=True)
    res = solve_mis(n, p, seed, time_limit, workers)
    print(f"{res['status']}, size={res['size']}, time={res['time_sec']:.1f}s")
    results.append(res)
    
# Summary
sizes = [r['size'] for r in results if r['size'] is not None]
avg_size = sum(sizes) / len(sizes) if sizes else 0
print("\nPer-seed results: ER(", n, ',', p, ') with time limit ', time_limit)
for r in results:
    print(f"  seed={r['seed']:>2}  status={r['status']:<8}  size={r['size']!s:<3}  time={r['time_sec']:.1f}s")
print(f"\nAverage independent set size (over feasible runs): {avg_size:.2f}")

Solving seed=0... UNKNOWN, size=None, time=54.2s
Solving seed=1... UNKNOWN, size=None, time=55.2s
Solving seed=2... UNKNOWN, size=None, time=54.0s
Solving seed=3... UNKNOWN, size=None, time=54.2s
Solving seed=4... UNKNOWN, size=None, time=53.4s
Solving seed=5... UNKNOWN, size=None, time=53.3s
Solving seed=6... UNKNOWN, size=None, time=53.7s
Solving seed=7... UNKNOWN, size=None, time=53.6s
Solving seed=8... UNKNOWN, size=None, time=53.6s
Solving seed=9... UNKNOWN, size=None, time=54.7s

Per-seed results: ER( 5000 , 0.5 ) with time limit  50
  seed= 0  status=UNKNOWN   size=None  time=54.2s
  seed= 1  status=UNKNOWN   size=None  time=55.2s
  seed= 2  status=UNKNOWN   size=None  time=54.0s
  seed= 3  status=UNKNOWN   size=None  time=54.2s
  seed= 4  status=UNKNOWN   size=None  time=53.4s
  seed= 5  status=UNKNOWN   size=None  time=53.3s
  seed= 6  status=UNKNOWN   size=None  time=53.7s
  seed= 7  status=UNKNOWN   size=None  time=53.6s
  seed= 8  status=UNKNOWN   size=None  time=53.6s
  se

Gurobi

In [5]:
def solve_mis_gurobi(n, p, seed, time_limit=10, verbose=False):
    """Builds and solves MIS on G(n,p) using Gurobi, returns result dict."""
    # Generate graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    # Create Gurobi model
    model = gp.Model(f"mis_seed_{seed}")
    # Add binary variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    # Edge constraints
    for u, v in G.edges():
        model.addConstr(x[u] + x[v] <= 1)
    # Objective
    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)
    # Parameters
    model.Params.TimeLimit = time_limit
    if not verbose:
        model.Params.OutputFlag = 0
    # Solve
    start = time.time()
    model.optimize()
    elapsed = time.time() - start
    # Extract result
    status = model.Status
    size = None
    if status in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        size = sum(1 for i in range(n) if x[i].X > 0.5)
    status_str = (
        "OPTIMAL" if status == GRB.OPTIMAL
        else "TIME_LIMIT" if status == GRB.TIME_LIMIT
        else str(status)
    )
    return {
        'seed': seed,
        'status': status_str,
        'size': size,
        'time_sec': elapsed
    }

# Batch execution over seeds 0–9
verbose = False

results = []
for seed in range(instances):
    print(f"Solving seed={seed}...", end=' ', flush=True)
    res = solve_mis_gurobi(n, p, seed, time_limit, verbose)
    print(f"{res['status']}, size={res['size']}, time={res['time_sec']:.1f}s")
    results.append(res)

# Summary statistics
sizes = [r['size'] for r in results if r['size'] is not None]
avg_size = sum(sizes) / len(sizes) if sizes else 0

# print("\nPer-seed results:")
print("\nPer-seed results: ER(", n, ',', p, ') with time limit ', time_limit)
for r in results:
    print(f"  seed={r['seed']:>2}  status={r['status']:<10}  size={r['size']!s:<3}  time={r['time_sec']:.1f}s")

print(f"\nAverage independent set size (over feasible runs): {avg_size:.2f}")

Solving seed=0... 

NameError: name 'gp' is not defined

In [4]:
def solve_mis_pulp_gurobi(n, p, seed, time_limit=10, verbose=False,
                          prefer_cmd=True, gurobi_path=None, extra_params=None):
    """
    Builds and solves MIS on G(n,p) using PuLP with Gurobi, returns result dict.

    prefer_cmd=True  -> use GUROBI_CMD (calls gurobi_cl), good for HPC module setups
    prefer_cmd=False -> use GUROBI (requires gurobipy available in the env)
    gurobi_path: optional path to gurobi_cl for GUROBI_CMD
    extra_params: dict of extra Gurobi params, e.g. {"MIPGap": 0.01}
    """
    extra_params = extra_params or {}

    # --- Generate graph
    G = nx.gnp_random_graph(n, p, seed=seed)

    # --- Build model
    prob = pl.LpProblem(f"mis_seed_{seed}", pl.LpMaximize)

    # Binary vars x_i in {0,1}
    x = {i: pl.LpVariable(f"x_{i}", lowBound=0, upBound=1, cat=pl.LpBinary) for i in range(n)}

    # Edge constraints: x_u + x_v <= 1
    for u, v in G.edges():
        prob += x[u] + x[v] <= 1

    # Objective: maximize sum_i x_i
    prob += pl.lpSum(x[i] for i in range(n))

    # --- Choose solver (Gurobi via PuLP)
    solver = None
    if prefer_cmd:
        # GUROBI_CMD → uses gurobi_cl (good for license/site-managed installs)
        # Time limit passed via the "TimeLimit" parameter.
        # You can add more params via extra_params.
        options = [("TimeLimit", time_limit)] + [(k, v) for k, v in extra_params.items()]
        solver = pl.GUROBI_CMD(path=gurobi_path, msg=verbose, options=options)
    else:
        # GUROBI (Python API via gurobipy)
        # timeLimit argument is accepted by PuLP GUROBI; also push extra params.
        solver = pl.GUROBI(msg=verbose, timeLimit=time_limit)
        for k, v in extra_params.items():
            solver.options[k] = v

    # --- Solve
    start = time.time()
    prob.solve(solver)
    elapsed = time.time() - start

    # --- Extract result
    pulp_status = pl.LpStatus[prob.status]
    # Heuristic: if not proven optimal but values exist, likely a feasible incumbent (often time limit)
    has_values = any(v.varValue is not None for v in x.values())
    if pulp_status == "Optimal":
        status_str = "OPTIMAL"
    elif has_values:
        # PuLP doesn't always propagate TIME_LIMIT as a distinct status for CMD; mark as such if feasible sol exists.
        status_str = "TIME_LIMIT"
    else:
        status_str = pulp_status.upper()

    size = None
    if has_values:
        size = sum(1 for i in range(n) if (x[i].varValue or 0.0) > 0.5)

    return {
        'seed': seed,
        'status': status_str,
        'size': size,
        'time_sec': elapsed
    }


# ================= Example batch run (matches your printing style) =================
if __name__ == "__main__":
    n = 500
    p = 0.5
    time_limit = 10
    instances = 10  # number of seeds (0..instances-1)
    verbose = False

    # prefer_cmd=True to use gurobi_cl via site module; set gurobi_path if needed (often not required if on $PATH)
    prefer_cmd = True
    gurobi_path = None  # e.g., "/path/to/gurobi_cl" if needed
    extra_params = {}   # e.g., {"MIPGap": 0.01}

    results = []
    for seed in range(instances):
        print(f"Solving seed={seed}...", end=' ', flush=True)
        res = solve_mis_pulp_gurobi(n, p, seed, time_limit, verbose,
                                    prefer_cmd=prefer_cmd, gurobi_path=gurobi_path,
                                    extra_params=extra_params)
        print(f"{res['status']}, size={res['size']}, time={res['time_sec']:.1f}s")
        results.append(res)

    sizes = [r['size'] for r in results if r['size'] is not None]
    avg_size = sum(sizes) / len(sizes) if sizes else 0

    print("\nPer-seed results: ER(", n, ',', p, ') with time limit ', time_limit)
    for r in results:
        print(f"  seed={r['seed']:>2}  status={r['status']:<10}  size={str(r['size']):<3}  time={r['time_sec']:.1f}s")

    print(f"\nAverage independent set size (over feasible runs): {avg_size:.2f}")


Solving seed=0... 

PulpSolverError: PuLP: cannot execute gurobi_cl

pCQO (MGD)

In [ ]:
# ===== problem & method settings =====
gamma_c = 1.01
beta    = 0.8
alpha   = 0.0001
ITERATION_T = m_it      # max inner iterations per init

exploration_parameter_eta = 2.0
covariance_matrix = exploration_parameter_eta * torch.eye(n)

# time budget per seed (seconds)
TIME_LIMIT = time_limit

# which seeds to run
SEEDS = range(instances)      # e.g. 0–9; expand as desired

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

# store best per seed
best_per_seed = []

for seed in SEEDS:
    # build ER graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    A     = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(G)).todense(), dtype=torch.float32)

    # compute gamma from complement degrees
    max_deg_c = max(dict(nx.complement(G).degree()).values())
    gamma = 2 + max_deg_c

    # degree-based init vector d
    degs = dict(G.degree())
    max_deg = max(degs.values())
    d = torch.tensor([1 - degs[i]/max_deg for i in range(n)], dtype=torch.float32)
    d = d / d.max()

    print(f"\nSeed {seed}: running MGD for up to {TIME_LIMIT}s")

    seed_start = time.time()
    trial = 0
    best_size = 0

    while time.time() - seed_start < TIME_LIMIT:
        # prepare initialization
        if trial == 0:
            x = d.clone()
        else:
            torch.manual_seed((seed << 16) + trial)
            x = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x.clamp_(0, 1)

        v = torch.zeros(n)  # velocity
        # run momentum gradient descent
        for it in range(ITERATION_T):
            grad = -torch.ones(n) + (gamma * A - gamma_c * A_bar) @ x
            v = beta * v + alpha * grad
            x = torch.clamp(x - v, 0, 1)

            checker, MIS = MIS_checker_efficient_3(x, A, A_bar)
            if checker:
                size = MIS.numel()
                if size > best_size:
                    best_size = size
                # print(f" trial {trial:2d} iters {it+1:4d} → MIS size {size}")
                break
        else:
            # reached ITERATION_T without finding MIS
            # print(f" trial {trial:2d} iters {ITERATION_T:4d} → no MIS")
            pass

        trial += 1

    print(f"→ Seed {seed} best MIS size = {best_size}")
    best_per_seed.append(best_size)

# overall average
avg_best = np.mean(best_per_seed)
print(f"\nAverage of best MIS size across seeds: {avg_best:.2f}")


Seed 0: running MGD for up to 50s
→ Seed 0 best MIS size = 13

Seed 1: running MGD for up to 50s
→ Seed 1 best MIS size = 14

Seed 2: running MGD for up to 50s
→ Seed 2 best MIS size = 13

Seed 3: running MGD for up to 50s
→ Seed 3 best MIS size = 14

Seed 4: running MGD for up to 50s
→ Seed 4 best MIS size = 13

Seed 5: running MGD for up to 50s
→ Seed 5 best MIS size = 14

Seed 6: running MGD for up to 50s
→ Seed 6 best MIS size = 12

Seed 7: running MGD for up to 50s
→ Seed 7 best MIS size = 13

Seed 8: running MGD for up to 50s
→ Seed 8 best MIS size = 13

Seed 9: running MGD for up to 50s
→ Seed 9 best MIS size = 14

Average of best MIS size across seeds: 13.30


Newton

In [5]:
# Fixed method parameters
gamma_c = 1
exploration_parameter_eta = 2.0
max_iters = 300
tol = 1e-8
damping0 = 1e-3
ls_iters = 10
alpha0 = 900   # fixed learning rate
beta = 0.6    # fixed backtracking scale

# Instead of N_TRIALS, run up to this many seconds per seed:
TIME_LIMIT = time_limit

# Functions
def objective(x, H):
    return -x.sum() + 0.5 * x @ (H @ x)

def gradient(x, H):
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def projected_newton(H, A, A_bar, x0):
    x = x0.clone()
    damping = damping0
    I = torch.eye(n, dtype=H.dtype)
    H_damped = H + damping * I
    history = []
    is_MIS = False
    MIS = None

    for it in range(max_iters):
        g = gradient(x, H)
        g_norm = g.norm().item()
        f0 = objective(x, H).item()
        history.append((it, f0, g_norm))
        if g_norm < tol:
            break

        try:
            p = torch.linalg.solve(H_damped, g)
        except RuntimeError:
            damping *= 10
            H_damped = H + damping * I
            continue

        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x - alpha * p, 0.0, 1.0)
            if objective(x_trial, H).item() < f0:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping *= 10

        damping = max(damping * 0.5, 1e-7)
        H_damped = H + damping * I

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# Main batch testing over seeds 0-9 with time‐limited trials
summaries = []
for seed in range(instances):
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + max(degrees_c.values())
    H = gamma * A - gamma_c * A_bar

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.tensor([1 - degrees[i]/max_deg for i in range(n)], dtype=torch.float32)
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run as many trials as will fit in TIME_LIMIT
    trial = 0
    trial_stats = []
    print(f"ER({n}, {p}, seed {seed}): Newton with MIS-check early exit, up to {TIME_LIMIT}s")
    seed_start = time.time()
    # print(f"\nSeed {seed}: running projected_newton for up to {TIME_LIMIT}s…")
    while True:
        # stop when time budget exceeded
        if time.time() - seed_start > TIME_LIMIT:
            break

        # prepare init
        if trial == 0:
            x0 = d_init.clone()
        else:
            torch.manual_seed((seed << 16) + trial)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0.clamp_(0.0, 1.0)

        # run Newton
        t0 = time.time()
        x_opt, history, is_MIS, MIS = projected_newton(H, A, A_bar, x0)
        elapsed = time.time() - t0

        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': MIS.numel() if is_MIS else 0
        })
        # print(f" trial {trial:2d} → iters {len(history):4d}, time {elapsed:7.4f}s, MIS_size {trial_stats[-1]['MIS_size']}")
        trial += 1

    # summarize for this seed
    df = pd.DataFrame(trial_stats)
    if not df.empty:
        summaries.append({
            'seed': seed,
            'n_trials':   len(df),
            'max_MIS_size': int(df.MIS_size.max()),
            'mean_iters': df.iters.mean(),
            'mean_time':  df.time.mean()
        })
    else:
        summaries.append({
            'seed': seed,
            'n_trials':   0,
            'max_MIS_size': 0,
            'mean_iters': 0.0,
            'mean_time':  0.0
        })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print("\nDamped Projected Newton (time‐limited) summary:")
print(df_summary)

avg_best = df_summary['max_MIS_size'].mean()
print(f"ER({n}, {p}): Newton with MIS-check early exit, up to {TIME_LIMIT}s")
print(f"\nAverage of best MIS size across seeds: {avg_best:.2f}")

ER(5000, 0.5, seed 0): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 1): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 2): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 3): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 4): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 5): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 6): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 7): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 8): Newton with MIS-check early exit, up to 50s
ER(5000, 0.5, seed 9): Newton with MIS-check early exit, up to 50s

Damped Projected Newton (time‐limited) summary:
   seed  n_trials  max_MIS_size  mean_iters   mean_time
0     0         1            13       280.0  172.358253
1     1         1             0       300.0  228.288355
2     2         1             0       300.0  226.139089
3     3         1             0       300.0  228.103144
4

Scipy's L-BFGS-B

In [6]:
# ===== your existing helper stays the same =====
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1 * (
        -torch.ones(n_local) + (n_local * adjacency_matrix_tensor) @ X_torch_binarized
    )
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

# ===== experiment settings =====
gamma_c = 1
beta = 0.8
alpha = 0.0001
ITERATION_T = m_it

# Instead of a fixed number of trials, run until this many seconds per seed:
TIME_LIMIT = time_limit  # seconds

SEEDS = range(instances)   # <- run 50 seeds

# for across-seed summary
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []

for seed in SEEDS:
    # ---- build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    num_components = nx.number_connected_components(G)
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", num_components)

    A = nx.adjacency_matrix(G).todense()
    A_t = torch.tensor(A, dtype=torch.float32)
    Acomp = nx.adjacency_matrix(complement_G).todense()
    Acomp_t = torch.tensor(Acomp, dtype=torch.float32)

    # gamma depends on complement degree
    max_degree = max(dict(complement_G.degree()).values())
    gamma = 2 + max_degree

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values())
    d = torch.tensor([1 - (degrees[i] / max_degree_node) for i in range(n)], dtype=torch.float32)
    d = d / d.max()

    # H and smooth objective/grad for SciPy
    # H_np = (gamma * A_t).numpy()
    H_np = (gamma * A_t - gamma_c * Acomp_t).numpy()

    def f_np(x):
        return -np.sum(x) + 0.5 * x.dot(H_np.dot(x))

    def grad_np(x):
        return -np.ones_like(x) + H_np.dot(x)

    bounds = [(0.0, 1.0)] * n

    # per-seed collectors
    MIS_size, Iter_NO, Run_time = [], [], []

    exploration_parameter_eta = 2.0
    covariance_matrix = exploration_parameter_eta * torch.eye(n)

    print(f"ER({n}, {p}, seed={seed}): L-BFGS-B with MIS-check early exit, up to {TIME_LIMIT}s")

    seed_start_time = time.time()
    trial = 0
    while True:
        # Check time budget
        if time.time() - seed_start_time > TIME_LIMIT:
            break

        # prepare init
        if trial == 0:
            x0_np = d.numpy()
        else:
            torch.manual_seed((seed << 16) + trial)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x0_np = noise.numpy()

        iter_counter = {"count": 0}
        last_x = {"xk": None}

        def lbfgs_callback(xk):
            iter_counter["count"] += 1
            last_x["xk"] = xk.copy()
            x_t = torch.tensor(xk, dtype=torch.float32)
            checker, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if checker:
                raise EarlyExit()

        start_time = time.time()
        try:
            res = minimize(
                fun      = f_np,
                x0       = x0_np,
                method   = "L-BFGS-B",
                jac      = grad_np,
                bounds   = bounds,
                callback = lbfgs_callback,
                options  = {
                    "maxiter": ITERATION_T,
                    "gtol":    0.0,
                    "ftol":    0.0,
                }
            )
            total_iters = res.nit
            x_final_np = res.x
        except EarlyExit:
            total_iters = iter_counter["count"]
            x_final_np = last_x["xk"]

        x_final = torch.tensor(x_final_np, dtype=torch.float32)
        checker, mis = MIS_checker_efficient_3(x_final, A_t, Acomp_t)

        elapsed = time.time() - start_time

        if mis is not None:
            # print(f" trial {trial:2d} → MIS size {len(mis):3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
            MIS_size.append(len(mis))
            Iter_NO.append(total_iters)
            Run_time.append(elapsed)
        else:
            # print(f" trial {trial:2d} → no MIS (iters {total_iters:4d}, time {elapsed:7.4f}s)")
            MIS_size.append(0)
            Iter_NO.append(total_iters)
            Run_time.append(elapsed)
            

        trial += 1

    # per-seed summary
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = mis_sizes.max()
        best_run  = mis_sizes.argmax()
        mean_rt   = runtimes.mean()
        mean_it   = iter_counts.mean()
        p2_5_rt, p97_5_rt = np.percentile(runtimes, [2.5, 97.5])
        p2_5_it, p97_5_it = np.percentile(iter_counts, [2.5, 97.5])
        
        '''print(f"— Seed {seed} summary —")
        print(f"Best MIS size: {best_size} (trial {best_run})")
        print(f"Runtime (s): mean={mean_rt:.4f}, min={runtimes.min():.4f}, max={runtimes.max():.4f}, "
              f"std={runtimes.std(ddof=1):.4f}, 95% CI=({p2_5_rt:.4f}, {p97_5_rt:.4f})")
        print(f"Iters: mean={mean_it:.1f}, min={iter_counts.min()}, max={iter_counts.max()}, "
              f"std={iter_counts.std(ddof=1):.1f}, 95% CI=({p2_5_it:.1f}, {p97_5_it:.1f})")'''

        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(int(best_run))
        mean_runtime_per_seed.append(float(mean_rt))
        mean_iters_per_seed.append(float(mean_it))
    else:
        print(f"— Seed {seed} summary — no MIS flagged by checker.")
        '''best_size_per_seed.append(None)
        best_trial_per_seed.append(None)
        mean_runtime_per_seed.append(None)
        mean_iters_per_seed.append(None)'''

# across-seeds summary
print(f"ER({n}, {p}): L-BFGS-B with MIS-check early exit, up to {TIME_LIMIT}s")
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("\n=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    # print(f"Best MIS size: max={vals.max()}, min={vals.min()}, mean={vals.mean():.2f}, std={vals.std(ddof=1):.2f}")
    print(f"Mean MIS size: {vals.mean():.2f}")


================== Seed 0 ==================
Number of connected components: 1
ER(5000, 0.5, seed=0): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 1 ==================
Number of connected components: 1
ER(5000, 0.5, seed=1): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 2 ==================
Number of connected components: 1
ER(5000, 0.5, seed=2): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 3 ==================
Number of connected components: 1
ER(5000, 0.5, seed=3): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 4 ==================
Number of connected components: 1
ER(5000, 0.5, seed=4): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 5 ==================
Number of connected components: 1
ER(5000, 0.5, seed=5): L-BFGS-B with MIS-check early exit, up to 50s

================== Seed 6 ==================
Number of connected components: 1
ER(5000, 0.5, seed=6): L-B

## Free Mask Settings ##

In [ ]:
import torch
import networkx as nx
import numpy as np
import time

# ===========================
# Helpers & MIS checker (same as your version)
# ===========================
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1 * (
        -torch.ones(n_local) + (n_local * adjacency_matrix_tensor) @ X_torch_binarized
    )
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

def project_box(x, lo=0.0, hi=1.0):
    return np.minimum(np.maximum(x, lo), hi)

def free_mask(x, g, lo=0.0, hi=1.0, eps=1e-8):
    inside = (x > lo + eps) & (x < hi - eps)
    at_lo_can_increase = (x <= lo + eps) & (g < 0)  # step -g increases x
    at_hi_can_decrease = (x >= hi - eps) & (g > 0)  # step -g decreases x
    return inside | at_lo_can_increase | at_hi_can_decrease

def lbfgs_two_loop_restricted(grad, S_list, Y_list, free):
    g = grad.copy()
    g[~free] = 0.0
    q = g.copy()
    alphas, rhos = [], []
    # reverse pass
    for s, y in reversed(list(zip(S_list, Y_list))):
        s_f = s.copy(); y_f = y.copy()
        s_f[~free] = 0.0; y_f[~free] = 0.0
        ys = float(y_f @ s_f)
        if ys <= 1e-16 or not np.isfinite(ys):
            alphas.append(0.0); rhos.append(1.0); continue
        rho = 1.0 / ys
        rhos.append(rho)
        alpha = rho * float(s_f @ q)
        alphas.append(alpha)
        q -= alpha * y_f
    # H0 scaling
    if len(S_list) > 0:
        y_last = Y_list[-1].copy(); s_last = S_list[-1].copy()
        y_last[~free] = 0.0; s_last[~free] = 0.0
        denom = float(y_last @ y_last)
        num   = float(s_last @ y_last)
        H0 = (num / denom) if (denom > 1e-16 and np.isfinite(num) and np.isfinite(denom)) else 1.0
        if H0 <= 1e-16: H0 = 1.0
    else:
        H0 = 1.0
    r = H0 * q
    # forward pass
    for (s, y), rho, alpha in zip(zip(S_list, Y_list), rhos[::-1], alphas[::-1]):
        s_f = s.copy(); y_f = y.copy()
        s_f[~free] = 0.0; y_f[~free] = 0.0
        ys = float(y_f @ s_f)
        if ys <= 1e-16 or not np.isfinite(ys):
            continue
        beta = rho * float(y_f @ r)
        r += s_f * (alpha - beta)
    p = -r
    p[~free] = 0.0
    return p

def backtracking_armijo_projected(f, x, fx, grad, p, proj_fn, c1=1e-4, tau=0.6, alpha0=1000.0, max_backtracks=80):
    gTp = float(grad @ p)
    if gTp >= 0:
        p = -grad
        gTp = float(grad @ p)
        if gTp >= 0:
            return 0.0, x, fx
    alpha = alpha0
    for _ in range(max_backtracks):
        x_trial = proj_fn(x + alpha * p)
        f_trial = f(x_trial)
        if f_trial <= fx + c1 * alpha * gTp:
            return alpha, x_trial, f_trial
        alpha *= tau
    return 0.0, x, fx

# ===========================
# Experiment settings (time-budgeted per seed)
# ===========================


gamma_c = 1.01             # we include gamma_c like your reference time-limit script
ITERATION_T = m_it

SEEDS = range(instances)       # adjust as you like

# L-BFGS knobs (same as your fast variant)
MEMORY_M = 10
ALPHA0   = 1000.0
TAU      = 0.6
PRINT_EVERY = 0         # set >0 for progress lines every k iters

# Across-seed collectors
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []
trials_per_seed = []    # how many inits fit in the time budget

for seed in SEEDS:
    # ---- build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    # adjacency (numpy + torch)
    A_np = np.asarray(nx.adjacency_matrix(G).astype(np.float32).todense(), dtype=np.float32)
    A_t  = torch.tensor(A_np)
    Acomp_np = np.asarray(nx.adjacency_matrix(complement_G).astype(np.float32).todense(), dtype=np.float32)
    Acomp_t  = torch.tensor(Acomp_np)

    # gamma from complement degree
    degrees_c = dict(complement_G.degree())
    max_degree_c = max(degrees_c.values()) if degrees_c else 0
    gamma = 2 + max_degree_c

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values()) if degrees else 1
    d = torch.zeros(n, dtype=torch.float32)
    for node, deg in G.degree():
        d[node] = 1.0 - (deg / max_degree_node if max_degree_node > 0 else 0.0)
    if d.max() > 0: d = d / d.max()

    # Quadratic model (match your SciPy time-limit reference: gamma*A - gamma_c*Acomp)
    H_np = (gamma * A_np - gamma_c * Acomp_np).astype(np.float32)

    def f_np(x):
        return float(-np.sum(x) + 0.5 * (x @ (H_np @ x)))

    def g_np(x):
        return (-np.ones_like(x) + H_np @ x).astype(np.float32)

    bounds_proj = lambda z: project_box(z, 0.0, 1.0)

    # per-seed stats
    MIS_size, Iter_NO, Run_time = [], [], []
    num_trials_done = 0

    # exploration noise
    exploration_parameter_eta = 2.0
    cov = exploration_parameter_eta * torch.eye(n)

    print(f"\n================== Seed {seed} ==================")
    print(f"ER({n}, {p}, seed={seed}): Handcrafted projected L-BFGS (no Cauchy), time budget = {time_limit}s")

    seed_start_time = time.time()
    trial = 0
    while True:
        # stop if time budget exceeded (strictly >, so the last finished trial counts)
        if time.time() - seed_start_time > time_limit:
            break

        # initialization
        if trial == 0:
            x0 = bounds_proj(d.numpy().astype(np.float32))
        else:
            torch.manual_seed((seed << 16) + trial)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(cov)))
            x0 = bounds_proj(noise.numpy().astype(np.float32))

        x = x0.copy()
        fx = f_np(x)
        g  = g_np(x)

        S_list, Y_list = [], []
        total_iters = 0
        start_time = time.time()
        mis_found = None

        for it in range(1, ITERATION_T + 1):
            total_iters = it

            # free set + direction
            free = free_mask(x, g, lo=0.0, hi=1.0, eps=1e-10)
            if not np.any(free):
                p_dir = -g
                p_dir[np.abs(p_dir) < 1e-12] = 0.0
            else:
                p_dir = lbfgs_two_loop_restricted(g, S_list, Y_list, free)
                if np.allclose(p_dir, 0.0):
                    p_dir = -g
                    p_dir[~free] = 0.0

            # line search (projected Armijo)
            alpha, x_new, f_new = backtracking_armijo_projected(
                f_np, x, fx, g, p_dir, bounds_proj,
                c1=1e-4, tau=TAU, alpha0=ALPHA0, max_backtracks=80
            )
            if alpha == 0.0:
                p_fb = -g; p_fb[~free] = 0.0
                if np.linalg.norm(p_fb) < 1e-14:
                    break
                x_new = bounds_proj(x + 1e-2 * p_fb)
                f_new = f_np(x_new)

            # MIS early exit
            chk, mis = MIS_checker_efficient_3(torch.tensor(x_new, dtype=torch.float32), A_t, Acomp_t)
            if chk:
                mis_found = mis
                x, fx = x_new, f_new
                if PRINT_EVERY and ((it % PRINT_EVERY) == 0 or it == 1):
                    print(f"  it {it:5d}: MIS found, f={fx:.4f}")
                break

            g_new = g_np(x_new)

            # curvature update
            s = (x_new - x).astype(np.float32)
            y = (g_new - g).astype(np.float32)
            ys = float(y @ s)
            if np.isfinite(ys) and ys > 1e-10:
                S_list.append(s); Y_list.append(y)
                if len(S_list) > MEMORY_M:
                    S_list.pop(0); Y_list.pop(0)

            x, fx, g = x_new, f_new, g_new

            if PRINT_EVERY and (it % PRINT_EVERY) == 0:
                pg_norm = np.linalg.norm(np.where(free, g, 0.0))
                print(f"  it {it:5d}: f={fx:.6f}, ||proj-grad||={pg_norm:.3e}")

        elapsed = time.time() - start_time
        num_trials_done += 1

        if mis_found is not None:
            size = int(len(mis_found))
            MIS_size.append(size); Iter_NO.append(total_iters); Run_time.append(elapsed)
        # else: no MIS found this trial → we just don’t append

        trial += 1

    trials_per_seed.append(num_trials_done)

    # per-seed summary (store only stats we need for across-seed mean)
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = int(mis_sizes.max())
        print("Best Size: ", best_size)
        best_run  = int(mis_sizes.argmax())
        mean_rt   = float(runtimes.mean())
        mean_it   = float(iter_counts.mean())

        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(best_run)
        mean_runtime_per_seed.append(mean_rt)
        mean_iters_per_seed.append(mean_it)
    else:
        # no MIS found within time budget for this seed → skip adding Nones
        pass

# across-seeds summary (match your SciPy style)
print(f"\nER({n}, {p}): Handcrafted projected L-BFGS (no Cauchy), time budget = {time_limit}s")
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    print(f"Mean MIS size: {vals.mean():.2f}")
    # If you also want to see how many trials ran (avg), uncomment:
    # print(f"Mean trials per seed (time-limited): {np.mean(trials_per_seed):.1f}")


================== Seed 0 ==================
ER(5000, 0.5, seed=0): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  13

================== Seed 1 ==================
ER(5000, 0.5, seed=1): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  14

================== Seed 2 ==================
ER(5000, 0.5, seed=2): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  13

================== Seed 3 ==================
ER(5000, 0.5, seed=3): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  13

================== Seed 4 ==================
ER(5000, 0.5, seed=4): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  14

================== Seed 5 ==================
ER(5000, 0.5, seed=5): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Size:  15

================== Seed 6 ==================
ER(5000, 0.5, seed=6): Handcrafted projected L-BFGS (no Cauchy), time budget = 50s
Best Si

In [ ]:
import torch
import networkx as nx
import numpy as np
import time

# ===========================
# Helpers & MIS checker
# ===========================
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(n_local) + (n_local*adjacency_matrix_tensor)@X_torch_binarized)
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

def project_box(x, lo=0.0, hi=1.0):
    return np.minimum(np.maximum(x, lo), hi)

# Build a "free set" mask like L-BFGS-B (who is allowed to move?)
def free_mask(x, g, lo=0.0, hi=1.0, eps=1e-8):
    # free if strictly inside OR at lower bound but gradient wants to go up (g<0 step -g goes up)
    # OR at upper bound but gradient wants to go down (g>0 step -g goes down)
    inside = (x > lo + eps) & (x < hi - eps)
    at_lo_can_increase = (x <= lo + eps) & (g < 0)   # step -g increases x
    at_hi_can_decrease = (x >= hi - eps) & (g > 0)   # step -g decreases x
    return inside | at_lo_can_increase | at_hi_can_decrease

# Two-loop recursion for (approx) -H_k * g restricted to free set
def lbfgs_two_loop_restricted(grad, S_list, Y_list, free):
    # Work only on free coordinates
    g = grad.copy()
    g[~free] = 0.0
    q = g.copy()
    alphas = []
    rhos = []
    # reverse pass
    for s, y in reversed(list(zip(S_list, Y_list))):
        s_f = s.copy(); y_f = y.copy()
        s_f[~free] = 0.0; y_f[~free] = 0.0
        ys = y_f @ s_f
        if ys <= 1e-16 or not np.isfinite(ys):
            alphas.append(0.0); rhos.append(1.0)
            continue
        rho = 1.0 / ys
        rhos.append(rho)
        alpha = rho * (s_f @ q)
        alphas.append(alpha)
        q -= alpha * y_f
    # H0 scaling
    if len(S_list) > 0:
        y_last = Y_list[-1].copy()
        s_last = S_list[-1].copy()
        y_last[~free] = 0.0; s_last[~free] = 0.0
        denom = y_last @ y_last
        num   = s_last @ y_last
        H0 = (num / denom) if (denom > 1e-16 and np.isfinite(num) and np.isfinite(denom)) else 1.0
        if H0 <= 1e-16: H0 = 1.0
    else:
        H0 = 1.0
    r = H0 * q
    # forward pass
    for (s, y), rho, alpha in zip(zip(S_list, Y_list), rhos[::-1], alphas[::-1]):
        s_f = s.copy(); y_f = y.copy()
        s_f[~free] = 0.0; y_f[~free] = 0.0
        ys = y_f @ s_f
        if ys <= 1e-16 or not np.isfinite(ys):
            continue
        beta = rho * (y_f @ r)
        r += s_f * (alpha - beta)
    p = -r
    # never move fixed coords
    p[~free] = 0.0
    return p

def backtracking_armijo_projected(f, x, fx, grad, p, proj_fn, c1=1e-4, tau=0.6, alpha0=1000.0, max_backtracks=80):
    # Use the directional derivative BEFORE projection (L-BFGS-B style)
    gTp = grad @ p
    # If not descent, fall back to steepest descent in free coordinates
    if gTp >= 0:
        p = -grad
        gTp = grad @ p
        if gTp >= 0:
            # gradient is zero or numerical; tiny step
            return 0.0, x, fx
    alpha = alpha0
    for _ in range(max_backtracks):
        x_trial = proj_fn(x + alpha * p)
        f_trial = f(x_trial)
        if f_trial <= fx + c1 * alpha * gTp:
            return alpha, x_trial, f_trial
        alpha *= tau
    return 0.0, x, fx

# ===========================
# Experiment settings
# ===========================
n = 500
p = 0.5
ITERATION_T = 15000
NUM_TRIALS = 10
SEEDS = range(50)

MEMORY_M = 10
ALPHA0   = 900   # aggressive initial step (your Newton used ~900)
TAU      = 0.6       # backtracking factor to match your Newton
PRINT_EVERY = 50     # progress prints

# Across-seed summary
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []

for seed in SEEDS:
    # ---- Build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    num_components = nx.number_connected_components(G)
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", num_components)

    A = nx.adjacency_matrix(G).astype(np.float32)
    A_np = np.asarray(A.todense(), dtype=np.float32)
    A_t = torch.tensor(A_np)

    Acomp = nx.adjacency_matrix(complement_G).astype(np.float32)
    Acomp_np = np.asarray(Acomp.todense(), dtype=np.float32)
    Acomp_t = torch.tensor(Acomp_np)

    # gamma based on complement degree
    degrees_c = dict(complement_G.degree())
    max_degree_c = max(degrees_c.values()) if degrees_c else 0
    gamma = 2 + max_degree_c

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values()) if degrees else 1
    d = torch.zeros(n, dtype=torch.float32)
    for node, deg in G.degree():
        d[node] = 1.0 - (deg / max_degree_node if max_degree_node > 0 else 0.0)
    if d.max() > 0: d = d / d.max()

    # Smooth quadratic (same as your SciPy definition with H = gamma*A)
    H_np = (gamma * A_np).astype(np.float32)

    def f_np(x):
        return float(-np.sum(x) + 0.5 * (x @ (H_np @ x)))

    def grad_np(x):
        return (-np.ones_like(x) + H_np @ x).astype(np.float32)

    bounds_proj = lambda z: project_box(z, 0.0, 1.0)

    # Per-seed collectors
    MIS_size, Iter_NO, Run_time = [], [], []

    # exploration noise
    exploration_parameter_eta = 2.0
    cov = exploration_parameter_eta * torch.eye(len(d))
    print(f"ER({n}, {p}, seed={seed}): Handcrafted projected L-BFGS-B (free-set) with MIS-check")

    for init in range(NUM_TRIALS):
        if init == 0:
            x0_np = d.numpy()
        else:
            torch.manual_seed((seed << 16) + init)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(cov)))
            x0_np = noise.numpy()

        x = bounds_proj(x0_np.astype(np.float32))
        fx = f_np(x)
        g  = grad_np(x)

        S_list, Y_list = [], []
        total_iters = 0
        start_time = time.time()
        mis_found = None

        for it in range(1, ITERATION_T + 1):
            total_iters = it

            # free set and direction
            free = free_mask(x, g, lo=0.0, hi=1.0, eps=1e-10)
            if not np.any(free):
                # everything clamped; try steepest descent anyway
                p_dir = -g
                p_dir[np.abs(p_dir) < 1e-12] = 0.0
            else:
                p_dir = lbfgs_two_loop_restricted(g, S_list, Y_list, free)

            # line search (projected Armijo)
            alpha, x_new, f_new = backtracking_armijo_projected(
                f_np, x, fx, g, p_dir, bounds_proj,
                c1=1e-4, tau=TAU, alpha0=ALPHA0, max_backtracks=80
            )

            # if line search failed to move, try a small steepest-descent nudge
            if alpha == 0.0:
                p_fallback = -g
                p_fallback[~free] = 0.0
                if np.linalg.norm(p_fallback) < 1e-14:
                    # nowhere to go
                    break
                alpha_small = 1e-2
                x_new = bounds_proj(x + alpha_small * p_fallback)
                f_new = f_np(x_new)

            # MIS early exit
            x_t = torch.tensor(x_new, dtype=torch.float32)
            chk, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if chk:
                mis_found = mis
                x, fx = x_new, f_new
                if (it % PRINT_EVERY) == 0 or it == 1:
                    print(f"  it {it:5d}: MIS found, f={fx:.4f}")
                break

            g_new = grad_np(x_new)

            # curvature pair with safeguard
            s = (x_new - x).astype(np.float32)
            y = (g_new - g).astype(np.float32)
            ys = float(y @ s)
            if np.isfinite(ys) and ys > 1e-10:
                S_list.append(s)
                Y_list.append(y)
                if len(S_list) > MEMORY_M:
                    S_list.pop(0); Y_list.pop(0)

            x, fx, g = x_new, f_new, g_new

            if (it % PRINT_EVERY) == 0:
                pg_norm = np.linalg.norm(np.where(free, g, 0.0))
                '''print(f"  it {it:5d}: f={fx:.6f}, ||proj-grad||={pg_norm:.3e}")'''

        elapsed = time.time() - start_time

        if mis_found is not None:
            size = int(len(mis_found))
            print(f"init {init:2d} → MIS size {size:3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
            MIS_size.append(size); Iter_NO.append(total_iters); Run_time.append(elapsed)
        else:
            # final check anyway
            x_t = torch.tensor(x, dtype=torch.float32)
            chk, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if chk:
                size = int(len(mis))
                print(f"init {init:2d} → MIS size {size:3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
                MIS_size.append(size); Iter_NO.append(total_iters); Run_time.append(elapsed)
            else:
                print(f"init {init:2d} → no MIS flagged (iters {total_iters}, time {elapsed:7.4f}s)")

    # Per-seed summary
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = int(mis_sizes.max())
        best_run  = int(mis_sizes.argmax())
        mean_rt   = float(runtimes.mean())
        mean_it   = float(iter_counts.mean())

        p2_5_rt, p97_5_rt = np.percentile(runtimes, [2.5, 97.5])
        p2_5_it, p97_5_it = np.percentile(iter_counts, [2.5, 97.5])

        print(f"— Seed {seed} summary —")
        print(f"Best MIS size: {best_size} (trial {best_run})")
        print(f"Runtime (s): mean={mean_rt:.4f}, min={runtimes.min():.4f}, max={runtimes.max():.4f}, "
              f"std={runtimes.std(ddof=1):.4f}, 95% CI=({p2_5_rt:.4f}, {p97_5_rt:.4f})")
        print(f"Iters: mean={mean_it:.1f}, min={iter_counts.min()}, max={iter_counts.max()}, "
              f"std={iter_counts.std(ddof=1):.1f}, 95% CI=({p2_5_it:.1f}, {p97_5_it:.1f})")

        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(best_run)
        mean_runtime_per_seed.append(mean_rt)
        mean_iters_per_seed.append(mean_it)
    else:
        print(f"— Seed {seed} summary — no MIS flagged by checker.")
        best_size_per_seed += [None]; best_trial_per_seed += [None]
        mean_runtime_per_seed += [None]; mean_iters_per_seed += [None]

# Across-seed summary
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("\n=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    print(f"Best MIS size: max={vals.max()}, min={vals.min()}, mean={vals.mean():.2f}, std={vals.std(ddof=1):.2f}")


================== Seed 0 ==================
Number of connected components: 1
ER(500, 0.5, seed=0): Handcrafted projected L-BFGS-B (free-set) with MIS-check
init  0 → MIS size  10, iters    24, time  0.0190s
init  1 → MIS size  10, iters    15, time  0.0114s
init  2 → MIS size   9, iters    18, time  0.0134s
init  3 → MIS size  10, iters    20, time  0.0132s
init  4 → MIS size  11, iters    14, time  0.0093s
init  5 → MIS size   8, iters    16, time  0.0132s
init  6 → MIS size  11, iters    22, time  0.0157s
init  7 → MIS size   8, iters    26, time  0.0199s
init  8 → MIS size  10, iters    22, time  0.0146s
init  9 → MIS size   9, iters    17, time  0.0117s
— Seed 0 summary —
Best MIS size: 11 (trial 4)
Runtime (s): mean=0.0141, min=0.0093, max=0.0199, std=0.0033, 95% CI=(0.0097, 0.0197)
Iters: mean=19.4, min=14, max=26, std=4.0, 95% CI=(14.2, 25.6)

================== Seed 1 ==================
Number of connected components: 1
ER(500, 0.5, seed=1): Handcrafted projected L-BFGS-B (f

In [19]:
import torch
import networkx as nx
import numpy as np
from scipy.optimize import minimize
import time

# ===== your existing helper stays the same =====
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(n_local) + (n_local*adjacency_matrix_tensor)@X_torch_binarized)
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

# ===== experiment settings (same as yours) =====
n = 500
p = 0.5
gamma_c = 1
beta = 0.8
alpha = 0.0001
ITERATION_T = 15000
NUM_TRIALS = 10
SEEDS = range(50)   # <- run 50 seeds

# for across-seed summary
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []

for seed in SEEDS:
    # ---- build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    num_components = nx.number_connected_components(G)
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", num_components)

    A = nx.adjacency_matrix(G)
    A_dense = A.todense()
    A_t = torch.tensor(A_dense, dtype=torch.float32)

    Acomp = nx.adjacency_matrix(complement_G)
    Acomp_dense = Acomp.todense()
    Acomp_t = torch.tensor(Acomp_dense, dtype=torch.float32)

    # gamma depends on complement degree
    degrees_c = dict(complement_G.degree())
    max_degree = max(degrees_c.values())
    gamma = 2 + max_degree

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values())
    d = torch.zeros(n)
    for node, degree in G.degree():
        d[node] = 1 - (degree / max_degree_node)
    d = d / d.max()

    # H and smooth objective/grad for SciPy
    H_np = (gamma * A_t - gamma_c * Acomp_t).numpy()
    # H_np = (gamma * A_t).numpy()

    def f_np(x):
        return -np.sum(x) + 0.5 * x.dot(H_np.dot(x))

    def grad_np(x):
        return -np.ones_like(x) + H_np.dot(x)

    bounds = [(0.0, 1.0)] * n

    # per-seed collectors
    MIS_size, Iter_NO, Run_time = [], [], []

    # same exploration noise as before
    exploration_parameter_eta = 2.0
    covariance_matrix = exploration_parameter_eta * torch.eye(len(d))

    print(f"ER({n}, {p}, seed={seed}): L-BFGS-B with MIS-check early exit")
    for init in range(NUM_TRIALS):
        if init == 0:
            x0_np = d.numpy()
        else:
            # make noise depend on (seed, init) so different seeds explore differently
            torch.manual_seed((seed << 16) + init)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x0_np = noise.numpy()

        iter_counter = {"count": 0}
        last_x = {"xk": None}

        def lbfgs_callback(xk):
            iter_counter["count"] += 1
            last_x["xk"] = xk.copy()
            x_t = torch.tensor(xk, dtype=torch.float32)
            checker, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if checker:
                raise EarlyExit()

        start_time = time.time()
        try:
            res = minimize(
                fun      = f_np,
                x0       = x0_np,
                method   = "L-BFGS-B",
                jac      = grad_np,
                bounds   = bounds,
                callback = lbfgs_callback,
                options  = {
                    "maxiter": ITERATION_T,
                    "gtol":    0.0,
                    "ftol":    0.0,
                    # "maxcor": 20,  # optional
                }
            )
            total_iters = res.nit
            x_final_np = res.x
        except EarlyExit:
            total_iters = iter_counter["count"]
            x_final_np = last_x["xk"]

        x_final = torch.tensor(x_final_np, dtype=torch.float32)
        checker, mis = MIS_checker_efficient_3(x_final, A_t, Acomp_t)

        elapsed = time.time() - start_time

        if mis is not None:
            print(f"init {init:2d} → MIS size {len(mis):3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
            MIS_size.append(len(mis))
            Iter_NO.append(total_iters)
            Run_time.append(elapsed)
        else:
            print(f"init {init:2d} → no MIS flagged by checker (iters {total_iters}, time {elapsed:7.4f}s)")

    # per-seed summary (printed before moving to next seed)
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = mis_sizes.max()
        best_run  = mis_sizes.argmax()
        mean_rt   = runtimes.mean()
        mean_it   = iter_counts.mean()

        p2_5_rt, p97_5_rt = np.percentile(runtimes, [2.5, 97.5])
        p2_5_it, p97_5_it = np.percentile(iter_counts, [2.5, 97.5])

        print(f"— Seed {seed} summary —")
        print(f"Best MIS size: {best_size} (trial {best_run})")
        print(f"Runtime (s): mean={mean_rt:.4f}, min={runtimes.min():.4f}, max={runtimes.max():.4f}, "
              f"std={runtimes.std(ddof=1):.4f}, 95% CI=({p2_5_rt:.4f}, {p97_5_rt:.4f})")
        print(f"Iters: mean={mean_it:.1f}, min={iter_counts.min()}, max={iter_counts.max()}, "
              f"std={iter_counts.std(ddof=1):.1f}, 95% CI=({p2_5_it:.1f}, {p97_5_it:.1f})")

        # store minimal across-seed info
        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(int(best_run))
        mean_runtime_per_seed.append(float(mean_rt))
        mean_iters_per_seed.append(float(mean_it))
    else:
        print(f"— Seed {seed} summary — no MIS flagged by checker.")
        best_size_per_seed.append(None)
        best_trial_per_seed.append(None)
        mean_runtime_per_seed.append(None)
        mean_iters_per_seed.append(None)

# optional: across-seeds summary
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("\n=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    print(f"Best MIS size: max={vals.max()}, min={vals.min()}, mean={vals.mean():.2f}, std={vals.std(ddof=1):.2f}")


================== Seed 0 ==================
Number of connected components: 1
ER(500, 0.5, seed=0): L-BFGS-B with MIS-check early exit
init  0 → MIS size  11, iters   149, time  2.3073s
init  1 → MIS size  10, iters   136, time  1.8556s
init  2 → MIS size  11, iters   135, time  1.8633s
init  3 → MIS size  10, iters   172, time  2.3247s
init  4 → MIS size   9, iters   184, time  2.4829s
init  5 → MIS size  12, iters   133, time  1.7853s
init  6 → MIS size  10, iters   161, time  2.1605s
init  7 → MIS size   9, iters   142, time  1.9173s
init  8 → MIS size   9, iters   180, time  2.4085s
init  9 → MIS size  13, iters   194, time  2.6212s
— Seed 0 summary —
Best MIS size: 13 (trial 9)
Runtime (s): mean=2.1727, min=1.7853, max=2.6212, std=0.2992, 95% CI=(1.8011, 2.5901)
Iters: mean=158.6, min=133, max=194, std=22.7, 95% CI=(133.4, 191.8)

================== Seed 1 ==================
Number of connected components: 1
ER(500, 0.5, seed=1): L-BFGS-B with MIS-check early exit
init  0 → MIS 

## Free Mask Newton ##

## Hard Fixing? ##

In [ ]:
# %% [markdown]
# Free-mask Projected Newton for relaxed MIS with fair timing & robust inits
# (PyTorch without `generator=` support on randn_like)

# %%
import torch
import networkx as nx
import numpy as np
import pandas as pd
import time

# ===========================
# Global params
# ===========================
n = 500
p = 0.5
SEEDS = range(10)
N_TRIALS = 10

# Newton params
max_iters = 1500
tol = 1e-8
damping0 = 1e-3
ls_iters_default = 40
alpha0_default = 1.0
beta_default = 0.5
c1_default = 1e-4

# Init sampling
sigma_init = 0.15  # small jitter to avoid clipping collapse

# ===========================
# Problem pieces
# ===========================
def objective(x: torch.Tensor, H: torch.Tensor) -> float:
    # f(x) = -1^T x + 1/2 x^T H x
    return float((-x.sum() + 0.5 * (x @ (H @ x))).item())

def gradient(x: torch.Tensor, H: torch.Tensor) -> torch.Tensor:
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X: torch.Tensor, A: torch.Tensor, A_bar: torch.Tensor):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def sample_init(d_init: torch.Tensor, sigma: float) -> torch.Tensor:
    # No generator kwarg; RNG determinism is controlled by torch.manual_seed(seed) once per seed
    x = d_init + sigma * torch.randn_like(d_init)
    return x.clamp_(0.0, 1.0)

# ===========================
# Free-mask Projected Newton (final)
# ===========================
@torch.no_grad()
def projected_newton_free(
    H: torch.Tensor, A: torch.Tensor, A_bar: torch.Tensor, x0: torch.Tensor,
    max_iters: int = max_iters, tol: float = tol,
    alpha0: float = alpha0_default, beta: float = beta_default,
    ls_iters: int = ls_iters_default, damping0: float = damping0,
    c1: float | None = c1_default
):
    """
    Projected Newton with L-BFGS-B-style free set.
    Solve (H_FF + λ I) d_F = g_F, take descent p = -d on F (zero elsewhere),
    update x_new = clamp(x + α p, 0, 1). Early exit on MIS.
    """
    x = x0.clone()
    damping = damping0
    history = []
    is_MIS = False
    MIS = None

    # Cache for current free set & damping
    cached_free_idx = None
    cached_damping = None
    L_cached = None  # cholesky(H_ff + damping I)
    eye_cache = None  # sized per free set

    for it in range(1, max_iters + 1):
        g = gradient(x, H)
        g_norm = float(g.norm().item())
        f0 = objective(x, H)
        history.append((it - 1, f0, g_norm))

        if g_norm < tol:
            break

        # L-BFGS-B free-set rule
        F_mask = ((x > 0.0 + 1e-10) & (x < 1.0 - 1e-10)) \
                 | ((x <= 0.0 + 1e-10) & (g < 0)) \
                 | ((x >= 1.0 - 1e-10) & (g > 0))
        free_idx = torch.nonzero(F_mask, as_tuple=False).flatten()

        # If nothing free, try a small projected steepest-descent nudge
        if free_idx.numel() == 0:
            p = -g
            p[(x <= 0.0 + 1e-12) & (p < 0)] = 0.0
            p[(x >= 1.0 - 1e-12) & (p > 0)] = 0.0
            if float(p.norm().item()) < 1e-14:
                break

            alpha = alpha0
            accepted = False
            for _ in range(ls_iters):
                x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
                if objective(x_trial, H) < f0:  # simple decrease
                    x = x_trial
                    accepted = True
                    break
                alpha *= beta
            if not accepted:
                break

            is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
            if is_MIS:
                break
            continue

        # Reduced Newton solve on free set: (H_ff + λI) d_f = g_f
        g_f = g[free_idx]

        need_refactor = (
            L_cached is None or
            cached_damping is None or
            cached_free_idx is None or
            (cached_damping != damping) or
            (cached_free_idx.numel() != free_idx.numel()) or
            (not torch.equal(cached_free_idx, free_idx))
        )

        if need_refactor:
            H_ff = H.index_select(0, free_idx).index_select(1, free_idx)
            H_ff = 0.5 * (H_ff + H_ff.T)  # symmetrize
            k = H_ff.shape[0]
            if (eye_cache is None) or (eye_cache.shape[0] != k):
                eye_cache = torch.eye(k, dtype=H_ff.dtype, device=H_ff.device)
            H_ff_damped = H_ff + damping * eye_cache

            # Build/repair Cholesky by adapting damping
            built = False
            for _ in range(10):
                try:
                    L_cached = torch.linalg.cholesky(H_ff_damped)
                    cached_free_idx = free_idx.clone()
                    cached_damping = damping
                    built = True
                    break
                except RuntimeError:
                    damping *= 10.0
                    H_ff_damped = H_ff + damping * eye_cache
            if not built:
                # fallback: projected steepest-descent on free set
                p = torch.zeros_like(g)
                p[free_idx] = -g_f
                alpha = alpha0
                accepted = False
                for _ in range(ls_iters):
                    x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
                    if objective(x_trial, H) < f0:
                        x = x_trial
                        accepted = True
                        break
                    alpha *= beta
                if not accepted:
                    damping = min(max(damping * 2.0, 1e-7), 1e7)
                else:
                    damping = max(damping * 0.5, 1e-7)
                is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
                if is_MIS:
                    break
                continue

        # Solve for d_f, then descent direction p = -d
        d_f = torch.cholesky_solve(g_f.unsqueeze(1), L_cached).squeeze(1)  # d_f = (H_ff + λI)^{-1} g_f
        p = torch.zeros_like(g)
        p[free_idx] = -d_f  # DESCENT direction

        # Guard: ensure descent slope on free set; if not, use -grad on free set
        slope = float((g[free_idx] * p[free_idx]).sum().item())
        if slope >= 0:
            p.zero_()
            p[free_idx] = -g_f

        # Projected backtracking (Armijo if c1 provided, else simple)
        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
            f_trial = objective(x_trial, H)
            if c1 is None:
                ok = (f_trial < f0)
            else:
                s = float((g[free_idx] * p[free_idx]).sum().item())
                ok = (s < 0) and (f_trial <= f0 + c1 * alpha * s)  # Armijo on free-set slope
            if ok:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping = min(max(damping * 10.0, 1e-7), 1e7)
            L_cached = None
            cached_free_idx = None
            cached_damping = None
        else:
            damping = max(damping * 0.5, 1e-7)

        # MIS early exit
        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# ===========================
# Experiment: seeds 0-49, 10 inits each
# ===========================
summaries = []
summaries_fair = []  # exclude init 0 from means (fair method timing)

for seed in SEEDS:
    # Build graph
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    # PSD option (recommended to reduce "all max-iters"):
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + (max(degrees_c.values()) if degrees_c else 0)
    H = gamma * A  # <-- PSD version
    # To use your original: H = gamma * A - 1.0 * A_bar

    # Degree-based init
    degrees = dict(g.degree())
    max_deg = max(degrees.values()) if degrees else 1
    d = torch.zeros(n, dtype=torch.float32)
    for node, deg in degrees.items():
        d[node] = 1.0 - (deg / max_deg if max_deg > 0 else 0.0)
    d_init = d / (d.max() if float(d.max().item()) > 0 else 1.0)

    # ---- Per-seed RNG & warm-up ----
    torch.manual_seed(seed)  # ensures distinct, deterministic draws per trial within this seed
    _ = projected_newton_free(H, A, A_bar, d_init)  # warm-up call (not timed)

    trial_stats = []
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", nx.number_connected_components(g))
    print(f"ER({n}, {p}, seed={seed}): Projected Newton (free set) with MIS-check")

    for init in range(N_TRIALS):
        if init == 0:
            x0 = d_init.clone()
        else:
            x0 = sample_init(d_init, sigma_init)

        start = time.time()
        x_opt, history, is_MIS, MIS = projected_newton_free(
            H, A, A_bar, x0,
            alpha0=alpha0_default, beta=beta_default,
            ls_iters=ls_iters_default, c1=c1_default
        )
        elapsed = time.time() - start

        MIS_size = int(MIS.numel()) if (is_MIS and hasattr(MIS, "numel")) else (int(MIS.item()) if is_MIS and getattr(MIS, "ndim", 1)==0 else 0)
        print(f"init {init:2d} → MIS size {MIS_size:3d}, iters {len(history):5d}, time {elapsed:7.4f}s")

        trial_stats.append({'iters': len(history), 'time': elapsed, 'MIS_size': MIS_size})

    df = pd.DataFrame(trial_stats)

    # Per-seed summaries
    best_mis = int(df.MIS_size.max())
    best_trial = int(df.MIS_size.idxmax())

    print(f"— Seed {seed} summary —")
    print(f"Best MIS size: {best_mis} (trial {best_trial})")
    print(f"Runtime (s): mean={df.time.mean():.4f}, min={df.time.min():.4f}, max={df.time.max():.4f}, "
          f"std={df.time.std(ddof=1):.4f}")
    print(f"Iters: mean={df.iters.mean():.1f}, min={df.iters.min()}, max={df.iters.max()}, "
          f"std={df.iters.std(ddof=1):.1f}")

    summaries.append({
        'seed': seed,
        'max_MIS_size': best_mis,
        'mean_iters': float(df.iters.mean()),
        'mean_time': float(df.time.mean())
    })

    # Fair (exclude init 0)
    df_fair = df.iloc[1:] if len(df) > 1 else df
    summaries_fair.append({
        'seed': seed,
        'max_MIS_size': best_mis,
        'mean_iters_fair': float(df_fair.iters.mean()),
        'mean_time_fair': float(df_fair.time.mean())
    })

# ===========================
# Aggregate summaries
# ===========================
df_summary = pd.DataFrame(summaries)
df_summary_fair = pd.DataFrame(summaries_fair)

print(f"\nProjected Newton (free set) on ER({n},{p}) over seeds 0-49, {N_TRIALS} trials each, "
      f"alpha0={alpha0_default}, beta={beta_default}, c1={c1_default}")
print("Overall means:")
print(df_summary)

print("\nFair means (exclude init 0 per seed):")
print(df_summary_fair)

# Optional: save CSVs
# df_summary.to_csv('newton_free_mask_summary_overall.csv', index=False)
# df_summary_fair.to_csv('newton_free_mask_summary_fair.csv', index=False)




================== Seed 0 ==================
Number of connected components: 1
ER(500, 0.5, seed=0): Projected Newton (free set) with MIS-check
init  0 → MIS size   8, iters    35, time  0.0229s
init  1 → MIS size   9, iters    35, time  0.0224s
init  2 → MIS size  10, iters    38, time  0.0241s
init  3 → MIS size  10, iters    38, time  0.0244s
init  4 → MIS size  10, iters    16, time  0.0162s
init  5 → MIS size  10, iters    15, time  0.0118s
init  6 → MIS size   9, iters    12, time  0.0104s
init  7 → MIS size  10, iters    13, time  0.0112s
init  8 → MIS size  11, iters    12, time  0.0102s
init  9 → MIS size  10, iters    16, time  0.0125s
— Seed 0 summary —
Best MIS size: 11 (trial 8)
Runtime (s): mean=0.0166, min=0.0102, max=0.0244, std=0.0061
Iters: mean=23.0, min=12, max=38, std=11.7

================== Seed 1 ==================
Number of connected components: 1
ER(500, 0.5, seed=1): Projected Newton (free set) with MIS-check
init  0 → MIS size  10, iters    15, time  0.0126

In [3]:
## Prelim Newton method ##

import torch
import networkx as nx
import numpy as np
import pandas as pd
import time

# Fixed method parameters
n = 500
p = 0.5
gamma_c = 1
exploration_parameter_eta = 2.0
max_iters = 1500
tol = 1e-8
damping0 = 1e-3
ls_iters = 10
alpha0 = 900   # fixed learning rate
beta = 0.6    # fixed backtracking scale
N_TRIALS = 10  # number of random initializations per graph

# Functions
def objective(x, H):
    return -x.sum() + 0.5 * x @ (H @ x)

def gradient(x, H):
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def projected_newton(H, A, A_bar, x0):
    x = x0.clone()
    damping = damping0
    I = torch.eye(n, dtype=H.dtype)
    H_damped = H + damping * I
    history = []
    is_MIS = False
    MIS = None

    for it in range(max_iters):
        g = gradient(x, H)
        g_norm = g.norm().item()
        f0 = objective(x, H).item()
        history.append((it, f0, g_norm))
        if g_norm < tol:
            break

        try:
            p = torch.linalg.solve(H_damped, g)
        except RuntimeError:
            damping *= 10
            H_damped = H + damping * I
            continue

        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x - alpha * p, 0.0, 1.0)
            if objective(x_trial, H).item() < f0:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping *= 10

        damping = max(damping * 0.5, 1e-7)
        H_damped = H + damping * I

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# Main batch testing over seeds 0-49
summaries = []
for seed in range(50):
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + max(degrees_c.values())
    H = gamma * A - gamma_c * A_bar

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.zeros(n)
    for node, deg in degrees.items():
        d[node] = 1 - deg / max_deg
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run multiple trials
    trial_stats = []
    for init in range(N_TRIALS):
        if init == 0:
            x0 = d_init.clone()
        else:
            torch.manual_seed((seed << 16) + init)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0 = torch.clamp(x0, 0.0, 1.0)

        start = time.time()
        x_opt, history, is_MIS, MIS = projected_newton(H, A, A_bar, x0)
        elapsed = time.time() - start
        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': MIS.numel() if is_MIS else 0
        })

    df = pd.DataFrame(trial_stats)
    summaries.append({
        'seed': seed,
        'max_MIS_size': int(df.MIS_size.max()),
        'mean_iters': df.iters.mean(),
        'mean_time': df.time.mean()
    })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print(f"Damped Projected Newton on ER({n},{p}) over seeds 0-49, 10 trials each, alpha={alpha0}, beta={beta}")
print(df_summary)

# Optionally save to CSV
# df_summary.to_csv('newton_seed_summary.csv', index=False)

Damped Projected Newton on ER(500,0.5) over seeds 0-49, 10 trials each, alpha=900, beta=0.6
    seed  max_MIS_size  mean_iters  mean_time
0      0            12        86.5   0.212316
1      1            11        72.5   0.156849
2      2            11        62.5   0.140046
3      3            11        66.5   0.142900
4      4            12        67.0   0.144347
5      5            10        81.2   0.172164
6      6            11        68.9   0.149604
7      7            12        96.8   0.278866
8      8            11        80.5   0.173906
9      9            10        85.2   0.181120
10    10            11        71.5   0.153822
11    11            11        74.7   0.161315
12    12            11       101.3   0.213111
13    13            11        67.2   0.144568
14    14            11        58.6   0.129787
15    15            11        69.5   0.150102
16    16            11        71.6   0.155345
17    17            12        66.5   0.142915
18    18            11        74.2

## Time Limit Testing ##

In [1]:
# %% [markdown]
# Time-limited Free-set Projected Newton for relaxed MIS (best-within-time per seed)

# %%
import torch
import networkx as nx
import numpy as np
import pandas as pd
import time

# ===========================
# Fixed method parameters
# ===========================
n = 5000
p = 0.5
gamma_c = 1
exploration_parameter_eta = 2.0

max_iters = 150
tol = 1e-8
damping0 = 1e-3

# Free-set Newton line-search defaults (robust)
alpha0 = 1.0     # starting step (smaller is safer with projection)
beta   = 0.5     # backtracking factor
ls_iters = 20    # allow deeper backtracking
c1 = 1e-4        # Armijo sufficient decrease (set to None to use simple decrease)

# Time budget per seed
TIME_LIMIT = 50  # seconds

# ===========================
# Objective, gradient, MIS checker
# ===========================
def objective(x: torch.Tensor, H: torch.Tensor) -> float:
    # f(x) = -1^T x + 1/2 x^T H x
    return float((-x.sum() + 0.5 * (x @ (H @ x))).item())

def gradient(x: torch.Tensor, H: torch.Tensor) -> torch.Tensor:
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X: torch.Tensor, A: torch.Tensor, A_bar: torch.Tensor):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

# ===========================
# Free-mask Projected Newton
# ===========================
@torch.no_grad()
def projected_newton_free(
    H: torch.Tensor, A: torch.Tensor, A_bar: torch.Tensor, x0: torch.Tensor,
    max_iters: int = max_iters, tol: float = tol,
    alpha0: float = alpha0, beta: float = beta,
    ls_iters: int = ls_iters, damping0: float = damping0,
    c1: float | None = c1
):
    """
    Projected Newton with L-BFGS-B-style free set.
    Solve (H_FF + λ I) d_F = g_F, take descent p = -d on F (zero elsewhere),
    update x_new = clamp(x + α p, 0, 1). Early exit on MIS.
    """
    x = x0.clone()
    damping = damping0
    history = []
    is_MIS = False
    MIS = None

    cached_free_idx = None
    cached_damping = None
    L_cached = None
    eye_cache = None

    for it in range(1, max_iters + 1):
        g = gradient(x, H)
        g_norm = float(g.norm().item())
        f0 = objective(x, H)
        history.append((it - 1, f0, g_norm))
        if g_norm < tol:
            break

        # L-BFGS-B free-set rule
        F_mask = ((x > 0.0 + 1e-10) & (x < 1.0 - 1e-10)) \
                 | ((x <= 0.0 + 1e-10) & (g < 0)) \
                 | ((x >= 1.0 - 1e-10) & (g > 0))
        free_idx = torch.nonzero(F_mask, as_tuple=False).flatten()

        # If nothing free, take small projected steepest-descent nudge
        if free_idx.numel() == 0:
            p = -g
            p[(x <= 0.0 + 1e-12) & (p < 0)] = 0.0
            p[(x >= 1.0 - 1e-12) & (p > 0)] = 0.0
            if float(p.norm().item()) < 1e-14:
                break

            alpha = alpha0
            accepted = False
            for _ in range(ls_iters):
                x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
                if objective(x_trial, H) < f0:
                    x = x_trial
                    accepted = True
                    break
                alpha *= beta
            if not accepted:
                break

            is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
            if is_MIS:
                break
            continue

        # Reduced Newton solve on free set: (H_ff + λI) d_f = g_f
        g_f = g[free_idx]

        need_refactor = (
            L_cached is None or
            cached_damping is None or
            cached_free_idx is None or
            (cached_damping != damping) or
            (cached_free_idx.numel() != free_idx.numel()) or
            (not torch.equal(cached_free_idx, free_idx))
        )

        if need_refactor:
            H_ff = H.index_select(0, free_idx).index_select(1, free_idx)
            H_ff = 0.5 * (H_ff + H_ff.T)
            k = H_ff.shape[0]
            if (eye_cache is None) or (eye_cache.shape[0] != k):
                eye_cache = torch.eye(k, dtype=H_ff.dtype, device=H_ff.device)
            H_ff_damped = H_ff + damping * eye_cache

            # Build/repair Cholesky by adapting damping
            built = False
            for _ in range(10):
                try:
                    L_cached = torch.linalg.cholesky(H_ff_damped)
                    cached_free_idx = free_idx.clone()
                    cached_damping = damping
                    built = True
                    break
                except RuntimeError:
                    damping *= 10.0
                    H_ff_damped = H_ff + damping * eye_cache
            if not built:
                # fallback: projected steepest-descent on free set
                p = torch.zeros_like(g)
                p[free_idx] = -g_f
                alpha = alpha0
                accepted = False
                for _ in range(ls_iters):
                    x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
                    if objective(x_trial, H) < f0:
                        x = x_trial
                        accepted = True
                        break
                    alpha *= beta
                if not accepted:
                    damping = min(max(damping * 2.0, 1e-7), 1e7)
                else:
                    damping = max(damping * 0.5, 1e-7)
                is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
                if is_MIS:
                    break
                continue

        # Solve for d_f, then descent direction p = -d
        d_f = torch.cholesky_solve(g_f.unsqueeze(1), L_cached).squeeze(1)
        p = torch.zeros_like(g); p[free_idx] = -d_f

        # Guard: ensure descent slope; if not, use -grad on free set
        slope = float((g[free_idx] * p[free_idx]).sum().item())
        if slope >= 0:
            p.zero_(); p[free_idx] = -g_f

        # Projected backtracking (Armijo if c1 else simple)
        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x + alpha * p, 0.0, 1.0)
            f_trial = objective(x_trial, H)
            if c1 is None:
                ok = (f_trial < f0)
            else:
                s = float((g[free_idx] * p[free_idx]).sum().item())
                ok = (s < 0) and (f_trial <= f0 + c1 * alpha * s)
            if ok:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping = min(max(damping * 10.0, 1e-7), 1e7)
            L_cached = None; cached_free_idx = None; cached_damping = None
        else:
            damping = max(damping * 0.5, 1e-7)

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# ===========================
# Main batch testing over seeds 0-9 with time-limited trials
# ===========================
summaries = []
for seed in range(10):
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H (PSD default for stability; switch to original by uncommenting the second line)
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + (max(degrees_c.values()) if degrees_c else 0)
    H = gamma * A             # PSD version (recommended)
    # H = gamma * A - gamma_c * A_bar  # original variant (may be indefinite on free blocks)

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.tensor([1 - degrees[i]/max_deg for i in range(n)], dtype=torch.float32)
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run as many trials as will fit in TIME_LIMIT
    trial = 0
    trial_stats = []
    print(f"ER({n}, {p}, seed {seed}): Free-set Projected Newton (time limit {TIME_LIMIT}s)")

    seed_start = time.time()
    while True:
        if time.time() - seed_start > TIME_LIMIT:
            break

        # prepare init
        if trial == 0:
            x0 = d_init.clone()
        else:
            # deterministic per (seed, trial); matches your baseline style
            torch.manual_seed((seed << 16) + trial)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0.clamp_(0.0, 1.0)

        # run Free-set Newton
        t0 = time.time()
        x_opt, history, is_MIS, MIS = projected_newton_free(
            H, A, A_bar, x0,
            alpha0=alpha0, beta=beta, ls_iters=ls_iters, c1=c1
        )
        elapsed = time.time() - t0

        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': int(MIS.numel()) if is_MIS else 0
        })
        # print(f" trial {trial:2d} → iters {len(history):4d}, time {elapsed:7.4f}s, MIS_size {trial_stats[-1]['MIS_size']}")
        trial += 1

    # summarize for this seed
    df = pd.DataFrame(trial_stats)
    if not df.empty:
        summaries.append({
            'seed': seed,
            'n_trials':   len(df),
            'max_MIS_size': int(df.MIS_size.max()),
            'mean_iters': float(df.iters.mean()),
            'mean_time':  float(df.time.mean())
        })
    else:
        summaries.append({
            'seed': seed,
            'n_trials':   0,
            'max_MIS_size': 0,
            'mean_iters': 0.0,
            'mean_time':  0.0
        })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print("\nFree-set Projected Newton (time-limited) summary:")
print(df_summary)

avg_best = df_summary['max_MIS_size'].mean()
print(f"\nER({n}, {p}): Free-set Newton with MIS-check early exit, up to {TIME_LIMIT}s per seed")
print(f"Average of best MIS size across seeds: {avg_best:.2f}")




KeyboardInterrupt: 

In [6]:
# With complement 

def build_upper_edge_list_from_graph(G: nx.Graph):
    """
    Store the edges of the undirected graph G as an edge list in upper triangular form (u < v).

    U, V: 1D torch.long tensors (length = number of edges |E|),
    representing the two endpoint indices (u, v) for each edge, where u < v.
    """
    edges = []
    for u, v in G.edges():
        if u == v: # Exclude self-loops
            continue
        if u > v:
            u, v = v, u
        edges.append((u, v))

    if not edges:
        # The case with no edges
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    U = torch.tensor([e[0] for e in edges], dtype=torch.long)
    V = torch.tensor([e[1] for e in edges], dtype=torch.long)
    return U, V


def sample_edge_sublist(U: torch.Tensor, V: torch.Tensor, p_keep: float):
    """
    Perform Bernoulli sampling for each "undirected edge" (u<v) to keep it with probability p_keep.
    Return the selected sub-edge list U_sub, V_sub (still in the u<v upper triangular form).
    """
    assert U.dtype == torch.long and V.dtype == torch.long
    assert U.shape == V.shape

    m = U.numel()
    mask = torch.rand(m) < p_keep
    idx = mask.nonzero(as_tuple=False).squeeze(1)

    if idx.numel() == 0:
        # When no edges are sampled, return an empty sublist.
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    return U.index_select(0, idx), V.index_select(0, idx)


def matrix_free_spmv(U_sub, V_sub, x, n, p_keep):
    """
    Matrix-Free Sparse Matrix-Vector Product
    y = (A_sub / p_keep) @ x
    """
    if U_sub.numel() == 0:
        return torch.zeros(n, dtype=x.dtype)

    y = torch.zeros(n, dtype=x.dtype)
    x_v = x.index_select(0, V_sub)
    x_u = x.index_select(0, U_sub)

    # y[u] += x[v] and y[v] += x[u]
    y.scatter_add_(0, U_sub, x_v)
    y.scatter_add_(0, V_sub, x_u)

    return y / p_keep

def partition_graph_edges(U: torch.Tensor, V: torch.Tensor, num_subgraphs: int):
    """
    Randomly shuffles the graph's edge list and partitions it into a specified number of subgraphs.

    Args:
        U (torch.Tensor): Tensor of source nodes for all edges.
        V (torch.Tensor): Tensor of destination nodes for all edges.
        num_subgraphs (int): The number of partitions to create.

    Returns:
        list: A list of tuples, where each tuple (U_sub, V_sub) represents an edge list
              of a subgraph partition.
    """
    num_edges = U.numel()
    # 处理没有边的情况
    if num_edges == 0:
        empty_subgraph = (torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long))
        return [empty_subgraph] * num_subgraphs

    # 创建一个随机的索引排列
    shuffled_indices = torch.randperm(num_edges)

    # 使用这个排列来打乱整个图的边列表
    U_shuffled = U[shuffled_indices]
    V_shuffled = V[shuffled_indices]

    # 将打乱后的边列表分割成 num_subgraphs个块
    U_chunks = torch.chunk(U_shuffled, num_subgraphs)
    V_chunks = torch.chunk(V_shuffled, num_subgraphs)

    # 将分块后的 (U, V) 对打包成列表并返回
    subgraph_partitions = list(zip(U_chunks, V_chunks))
    return subgraph_partitions


def check_maximal_independent_set(X_torch, n, edges_g_set):
    """
    Check whether a solution is a Maximal Independent Set based on an edge list (hash set).
    """
    # 1. Retrieve the candidate node set
    X_binarized = X_torch.bool()
    S_nodes_tensor = X_binarized.nonzero(as_tuple=False).squeeze(1)
    S_nodes_set = set(S_nodes_tensor.tolist())

    # 2. Check whether S is an independent set:
    # iterate through all node pairs in S, and if any pair is found in the edge hash set, then S is not an IS
    for u in S_nodes_tensor:
        for v in S_nodes_tensor:
            if u >= v: continue
            if (u.item(), v.item()) in edges_g_set:
                return False, None

    # 3. Check whether S is a maximal independent set:
    # iterate through all nodes w not in S, and check whether w can be added to S
    all_nodes = set(range(n))
    outside_nodes = all_nodes - S_nodes_set

    for w in outside_nodes:
        can_be_added = True
        for s_node in S_nodes_set:
            u, v = min(w, s_node), max(w, s_node)
            if (u, v) in edges_g_set:
                can_be_added = False
                break
        if can_be_added:
            # If there exists a node that can be added, then S is not maximal
            return False, None

    # If all checks are passed, then S is a Maximal Independent Set
    return True, S_nodes_tensor

#########################################################################
# Begin
#########################################################################
# node_list = [500, 1000, 2000, 3000, 5000]
node_list = [500, 1000]
# node_list = [2000]
p = 0.5

# num_subgraphs = 5
num_subgraphs = 10

TIME_LIMIT = 5
# TIME_LIMIT = 10

with open("sparse_partition_subgraph_with_complement.txt", "w") as f:
    for n in node_list:
        best_list = []
        conv_iters_all = []
        for seed in range(10):
            print(f"\n===== Running for n = {n}, seed = {seed} =====")
            f.write(f"Number of nodes n = {n}\n")
            f.write(f"Edge probability p = {p}\n")
            f.write(f"Graph Seed = {seed}\n")
            f.write("Results for each epoch:\n")

            G = nx.gnp_random_graph(n, p, seed=seed)

            # Completely remove all adjacency matrix constructions and use only edge lists.
            print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
            U, V = build_upper_edge_list_from_graph(G)

            # Precompute a hash set of edges for efficient lookup in the MIS checker
            edges_g_set = set(zip(U.tolist(), V.tolist()))

            # Construct the edge list of the complement graph, then discard the complement graph object
            complement_G = nx.complement(G)
            U_comp, V_comp = build_upper_edge_list_from_graph(complement_G)
            del complement_G  # Free up memory

            # --------------------------------------------------------------------------
            ### Here, we set gamma, gamma', and beta.
            gamma_c = 1.01

            # Compute the degree in the complement graph
            degrees_g = dict(G.degree())
            degrees_c = {node: (n - 1) - degree for node, degree in degrees_g.items()}
            max_degree = max(degrees_c.values()) if degrees_c else 0
            gamma = 2 + max_degree
            # --------------------------------------------------------------------------

            beta = 0.8
            alpha = 0.0001

            if (n == 3000 or n == 5000) and num_subgraphs == 10:
                alpha = 0.00001

            ITERATION_T = 150
            if n == 3000 or n == 5000:
                ITERATION_T = 300

            epoch = 50

            # Degree-based initialization
            degrees = dict(G.degree())
            max_degree_node = max(degrees.values()) if degrees else 1
            d = torch.zeros(n)
            for node, degree in G.degree():
                d[node] = 1 - (degree / max_degree_node)
            if max(d) > 0: d = d / max(d)

            input_velocity = torch.zeros(n)
            exploration_parameter_eta = 2.0
            covariance_matrix = exploration_parameter_eta * torch.eye(len(d))
            best_MIS_size = 0
            best_epoch = -1
            best_iter = -1
            conv_iters_seed = []

            # p_keep参数现在用于缩放梯度，其值等于子图中边的比例
            p_keep_for_scaling = 1.0 / num_subgraphs

            # ===============================================================
            # time limit
            start_time = time.time()

            init = -1

            while time.time() - start_time < TIME_LIMIT:
                init = init + 1
                if time.time() - start_time > TIME_LIMIT:
                    break

            # for init in range(epoch):
                if init == 0:
                    input_tensor = d.clone()
                else:
                    torch.manual_seed(init + seed)  # Ensure that each random initialization is different
                    input_tensor = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))

                input_velocity = torch.zeros(n)
                converged = False
                conv_iter = None
                MIS_size = 0

                # To get the subgraph
                # -----------------------------------------------------------------------
                subgraph_partitions = partition_graph_edges(U, V, num_subgraphs)
                subgraph_partitions_comp = partition_graph_edges(U_comp, V_comp, num_subgraphs)
                # -----------------------------------------------------------------------

                for iteration in range(ITERATION_T):
                    # 从预先生成的子图分区列表中循环选择当前迭代所用的子图
                    U_sub, V_sub = subgraph_partitions[iteration % num_subgraphs]
                    U_comp_sub, V_comp_sub = subgraph_partitions_comp[iteration % num_subgraphs]

                    # Compute gradient components using a matrix-free method
                    ax = matrix_free_spmv(U_sub, V_sub, input_tensor, n, p_keep_for_scaling)
                    acx = matrix_free_spmv(U_comp_sub, V_comp_sub, input_tensor, n, p_keep_for_scaling)

                    # Synthesize the final gradient
                    gradient = -torch.ones(n) + (gamma * ax - gamma_c * acx)
                    # gradient = -torch.ones(n) + gamma * ax
                    # ----------------------------------------

                    input_velocity = beta * input_velocity + alpha * gradient
                    input_tensor = torch.clamp(input_tensor - input_velocity, 0, 1)

                    # Call the new edge-list-based MIS checker
                    Checker, MIS = check_maximal_independent_set(input_tensor, n, edges_g_set)

                    if Checker:
                        MIS_size = MIS.numel()
                        conv_iter = iteration
                        converged = True
                        break

                if converged:
                    conv_iters_seed.append(conv_iter)
                    # f.write(
                    #     f"Epoch {init}, MIS size = {MIS_size}, converged at iteration = {conv_iter}\n"
                    # )
                    if MIS_size > best_MIS_size:
                        best_MIS_size = MIS_size
                        best_epoch = init
                        best_iter = conv_iter
                # else:
                #     f.write(f"Epoch {init}, No convergence (ran {ITERATION_T} iterations)\n")

            best_list.append(best_MIS_size)
            if conv_iters_seed:
                avg_conv_iter = np.mean(conv_iters_seed)
                conv_iters_all.extend(conv_iters_seed)
                f.write(f"\nAverage convergence iteration (this seed) = {avg_conv_iter:.2f}\n")
            else:
                f.write("\nNo epoch converged in this seed.\n")
            f.write(f"\nBest MIS size = {best_MIS_size}\n")
            f.write(f"\n=============================================\n")
            print(f"n={n}, seed={seed}, best MIS size = {best_MIS_size}")

        avg_best = np.mean(best_list) if best_list else 0
        if conv_iters_all:
            avg_conv_all = np.mean(conv_iters_all)
            f.write(
                f"\n=**********************************************************************************************************\n")
            f.write(f"Average Best MIS size over {len(best_list)} seeds = {avg_best:.2f}\n")
            f.write(f"Average convergence iteration over all converged epochs = {avg_conv_all:.2f}\n")
            f.write(
                f"=************************************************************************************************************\n\n")
        else:
            f.write("\nNo convergence across all seeds.\n")


===== Running for n = 500, seed = 0 =====
Graph has 500 nodes and 62475 edges.
n=500, seed=0, best MIS size = 12

===== Running for n = 500, seed = 1 =====
Graph has 500 nodes and 62440 edges.
n=500, seed=1, best MIS size = 13

===== Running for n = 500, seed = 2 =====
Graph has 500 nodes and 62205 edges.
n=500, seed=2, best MIS size = 12

===== Running for n = 500, seed = 3 =====
Graph has 500 nodes and 62556 edges.
n=500, seed=3, best MIS size = 12

===== Running for n = 500, seed = 4 =====
Graph has 500 nodes and 62558 edges.
n=500, seed=4, best MIS size = 12

===== Running for n = 500, seed = 5 =====
Graph has 500 nodes and 62598 edges.
n=500, seed=5, best MIS size = 11

===== Running for n = 500, seed = 6 =====
Graph has 500 nodes and 62272 edges.
n=500, seed=6, best MIS size = 12

===== Running for n = 500, seed = 7 =====
Graph has 500 nodes and 62560 edges.
n=500, seed=7, best MIS size = 13

===== Running for n = 500, seed = 8 =====
Graph has 500 nodes and 62663 edges.
n=500, s

In [17]:
# without complement

import torch
import networkx as nx
import numpy as np
import time
import psutil, os


def build_upper_edge_list_from_graph(G: nx.Graph):
    """
    Store the edges of the undirected graph G as an edge list in upper triangular form (u < v).

    U, V: 1D torch.long tensors (length = number of edges |E|),
    representing the two endpoint indices (u, v) for each edge, where u < v.
    """
    edges = []
    for u, v in G.edges():
        if u == v: # Exclude self-loops
            continue
        if u > v:
            u, v = v, u
        edges.append((u, v))

    if not edges:
        # The case with no edges
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    U = torch.tensor([e[0] for e in edges], dtype=torch.long)
    V = torch.tensor([e[1] for e in edges], dtype=torch.long)
    return U, V


def sample_edge_sublist(U: torch.Tensor, V: torch.Tensor, p_keep: float):
    """
    Perform Bernoulli sampling for each "undirected edge" (u<v) to keep it with probability p_keep.
    Return the selected sub-edge list U_sub, V_sub (still in the u<v upper triangular form).
    """
    assert U.dtype == torch.long and V.dtype == torch.long
    assert U.shape == V.shape

    m = U.numel()
    mask = torch.rand(m) < p_keep
    idx = mask.nonzero(as_tuple=False).squeeze(1)

    if idx.numel() == 0:
        # When no edges are sampled, return an empty sublist.
        return torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long)

    return U.index_select(0, idx), V.index_select(0, idx)


def matrix_free_spmv(U_sub, V_sub, x, n, p_keep):
    """
    Matrix-Free Sparse Matrix-Vector Product
    y = (A_sub / p_keep) @ x
    """
    if U_sub.numel() == 0:
        return torch.zeros(n, dtype=x.dtype)

    y = torch.zeros(n, dtype=x.dtype)
    x_v = x.index_select(0, V_sub)
    x_u = x.index_select(0, U_sub)

    # y[u] += x[v] and y[v] += x[u]
    y.scatter_add_(0, U_sub, x_v)
    y.scatter_add_(0, V_sub, x_u)

    return y / p_keep

def partition_graph_edges(U: torch.Tensor, V: torch.Tensor, num_subgraphs: int):
    """
    Randomly shuffles the graph's edge list and partitions it into a specified number of subgraphs.

    Args:
        U (torch.Tensor): Tensor of source nodes for all edges.
        V (torch.Tensor): Tensor of destination nodes for all edges.
        num_subgraphs (int): The number of partitions to create.

    Returns:
        list: A list of tuples, where each tuple (U_sub, V_sub) represents an edge list
              of a subgraph partition.
    """
    num_edges = U.numel()
    # 处理没有边的情况
    if num_edges == 0:
        empty_subgraph = (torch.empty(0, dtype=torch.long), torch.empty(0, dtype=torch.long))
        return [empty_subgraph] * num_subgraphs

    # 创建一个随机的索引排列
    shuffled_indices = torch.randperm(num_edges)

    # 使用这个排列来打乱整个图的边列表
    U_shuffled = U[shuffled_indices]
    V_shuffled = V[shuffled_indices]

    # 将打乱后的边列表分割成 num_subgraphs个块
    U_chunks = torch.chunk(U_shuffled, num_subgraphs)
    V_chunks = torch.chunk(V_shuffled, num_subgraphs)

    # 将分块后的 (U, V) 对打包成列表并返回
    subgraph_partitions = list(zip(U_chunks, V_chunks))
    return subgraph_partitions


def check_maximal_independent_set(X_torch, n, edges_g_set):
    """
    Check whether a solution is a Maximal Independent Set based on an edge list (hash set).
    """
    # 1. Retrieve the candidate node set
    X_binarized = X_torch.bool()
    S_nodes_tensor = X_binarized.nonzero(as_tuple=False).squeeze(1)
    S_nodes_set = set(S_nodes_tensor.tolist())

    # 2. Check whether S is an independent set:
    # iterate through all node pairs in S, and if any pair is found in the edge hash set, then S is not an IS
    for u in S_nodes_tensor:
        for v in S_nodes_tensor:
            if u >= v: continue
            if (u.item(), v.item()) in edges_g_set:
                return False, None

    # 3. Check whether S is a maximal independent set:
    # iterate through all nodes w not in S, and check whether w can be added to S
    all_nodes = set(range(n))
    outside_nodes = all_nodes - S_nodes_set

    for w in outside_nodes:
        can_be_added = True
        for s_node in S_nodes_set:
            u, v = min(w, s_node), max(w, s_node)
            if (u, v) in edges_g_set:
                can_be_added = False
                break
        if can_be_added:
            # If there exists a node that can be added, then S is not maximal
            return False, None

    # If all checks are passed, then S is a Maximal Independent Set
    return True, S_nodes_tensor

#########################################################################
# Begin
#########################################################################
# node_list = [500, 1000, 2000, 3000, 5000]
# node_list = [3000]
node_list = [500, 1000]
p = 0.5

num_subgraphs = 5
# num_subgraphs = 10

# TIME_LIMIT = 20
TIME_LIMIT = 5

with open("sparse_partition_subgraph_without_complement.txt", "w") as f:
    for n in node_list:
        best_list = []
        conv_iters_all = []
        for seed in range(10):
            print(f"\n===== Running for n = {n}, seed = {seed} =====")
            f.write(f"Number of nodes n = {n}\n")
            f.write(f"Edge probability p = {p}\n")
            f.write(f"Graph Seed = {seed}\n")
            f.write("Results for each epoch:\n")

            G = nx.gnp_random_graph(n, p, seed=seed)

            # Completely remove all adjacency matrix constructions and use only edge lists.
            print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
            U, V = build_upper_edge_list_from_graph(G)

            # Precompute a hash set of edges for efficient lookup in the MIS checker
            edges_g_set = set(zip(U.tolist(), V.tolist()))

            # Construct the edge list of the complement graph, then discard the complement graph object
            # complement_G = nx.complement(G)
            # U_comp, V_comp = build_upper_edge_list_from_graph(complement_G)
            # del complement_G  # Free up memory

            # --------------------------------------------------------------------------
            ### Here, we set gamma, gamma', and beta.
            gamma_c = 1.01

            # Compute the degree in the complement graph
            degrees_g = dict(G.degree())
            degrees_c = {node: (n - 1) - degree for node, degree in degrees_g.items()}
            max_degree = max(degrees_c.values()) if degrees_c else 0
            gamma = 2 + max_degree
            # --------------------------------------------------------------------------

            beta = 0.8
            alpha = 0.0001

            if (n == 3000 or n == 5000) and num_subgraphs == 10:
                alpha = 0.00001

            ITERATION_T = 150
            if n == 3000 or n == 5000:
                ITERATION_T = 300

            epoch = 50

            # Degree-based initialization
            degrees = dict(G.degree())
            max_degree_node = max(degrees.values()) if degrees else 1
            d = torch.zeros(n)
            for node, degree in G.degree():
                d[node] = 1 - (degree / max_degree_node)
            if max(d) > 0: d = d / max(d)

            input_velocity = torch.zeros(n)
            exploration_parameter_eta = 2.0
            covariance_matrix = exploration_parameter_eta * torch.eye(len(d))
            best_MIS_size = 0
            best_epoch = -1
            best_iter = -1
            conv_iters_seed = []

            # p_keep参数现在用于缩放梯度，其值等于子图中边的比例
            p_keep_for_scaling = 1.0 / num_subgraphs
            # ===============================================================
            # time limit
            start_time = time.time()

            init = -1

            while time.time() - start_time < TIME_LIMIT:
                init = init + 1
                if time.time() - start_time > TIME_LIMIT:
                    break

            # for init in range(epoch):
            #     elapsed_time = time.time() - start_time
            #     if elapsed_time > TIME_LIMIT:  # 超过时间限制就退出
            #         print(f"Time limit reached after {init} epochs (elapsed {elapsed_time:.2f}s)")
            #         break

                if init == 0:
                    input_tensor = d.clone()
                else:
                    torch.manual_seed(init + seed)  # Ensure that each random initialization is different
                    input_tensor = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))

                input_velocity = torch.zeros(n)
                converged = False
                conv_iter = None
                MIS_size = 0

                # To get the subgraph
                # -----------------------------------------------------------------------
                subgraph_partitions = partition_graph_edges(U, V, num_subgraphs)
                # -----------------------------------------------------------------------

                for iteration in range(ITERATION_T):

                    # 从预先生成的子图分区列表中循环选择当前迭代所用的子图
                    U_sub, V_sub = subgraph_partitions[iteration % num_subgraphs]

                    # Compute gradient components using a matrix-free method
                    ax = matrix_free_spmv(U_sub, V_sub, input_tensor, n, p_keep_for_scaling)
                    # acx = matrix_free_spmv(U_comp_sub, V_comp_sub, input_tensor, n, p_keep)

                    # Synthesize the final gradient
                    # gradient = -torch.ones(n) + (gamma * ax - gamma_c * acx)
                    gradient = -torch.ones(n) + gamma * ax
                    # ----------------------------------------

                    input_velocity = beta * input_velocity + alpha * gradient
                    input_tensor = torch.clamp(input_tensor - input_velocity, 0, 1)

                    # Call the new edge-list-based MIS checker
                    Checker, MIS = check_maximal_independent_set(input_tensor, n, edges_g_set)

                    if Checker:
                        MIS_size = MIS.numel()
                        conv_iter = iteration
                        converged = True
                        break


                if converged:
                    conv_iters_seed.append(conv_iter)
                    # f.write(
                    #     f"Epoch {init}, MIS size = {MIS_size}, converged at iteration = {conv_iter}\n"
                    # )
                    if MIS_size > best_MIS_size:
                        best_MIS_size = MIS_size
                        best_epoch = init
                        best_iter = conv_iter
                # else:
                #     f.write(f"Epoch {init}, No convergence (ran {ITERATION_T} iterations)\n")

            best_list.append(best_MIS_size)
            if conv_iters_seed:
                avg_conv_iter = np.mean(conv_iters_seed)
                conv_iters_all.extend(conv_iters_seed)
                f.write(f"\nAverage convergence iteration (this seed) = {avg_conv_iter:.2f}\n")
            else:
                f.write("\nNo epoch converged in this seed.\n")
            f.write(f"\nBest MIS size = {best_MIS_size}\n")
            f.write(f"\n=============================================\n")
            print(f"n={n}, seed={seed}, best MIS size = {best_MIS_size}")

        avg_best = np.mean(best_list) if best_list else 0
        if conv_iters_all:
            avg_conv_all = np.mean(conv_iters_all)
            f.write(
                f"\n=**********************************************************************************************************\n")
            f.write(f"Average Best MIS size over {len(best_list)} seeds = {avg_best:.2f}\n")
            f.write(f"Average convergence iteration over all converged epochs = {avg_conv_all:.2f}\n")
            f.write(
                f"=************************************************************************************************************\n\n")
        else:
            f.write("\nNo convergence across all seeds.\n")


===== Running for n = 500, seed = 0 =====
Graph has 500 nodes and 62475 edges.
n=500, seed=0, best MIS size = 11

===== Running for n = 500, seed = 1 =====
Graph has 500 nodes and 62440 edges.
n=500, seed=1, best MIS size = 11

===== Running for n = 500, seed = 2 =====
Graph has 500 nodes and 62205 edges.
n=500, seed=2, best MIS size = 11

===== Running for n = 500, seed = 3 =====
Graph has 500 nodes and 62556 edges.
n=500, seed=3, best MIS size = 12

===== Running for n = 500, seed = 4 =====
Graph has 500 nodes and 62558 edges.
n=500, seed=4, best MIS size = 11

===== Running for n = 500, seed = 5 =====
Graph has 500 nodes and 62598 edges.
n=500, seed=5, best MIS size = 12

===== Running for n = 500, seed = 6 =====
Graph has 500 nodes and 62272 edges.
n=500, seed=6, best MIS size = 11

===== Running for n = 500, seed = 7 =====
Graph has 500 nodes and 62560 edges.
n=500, seed=7, best MIS size = 11

===== Running for n = 500, seed = 8 =====
Graph has 500 nodes and 62663 edges.
n=500, s